# Loading and sorting the data

Todo: 
- Provide description that we are using combination of RealmType and Enclave
- Find Ambonmalities and graph
- Find and graph missing values

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/arison/Projects/Hz/datascience/iteration1/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
  File "/tmp/ipykernel_4436/1974400748.py", line 1, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/arison/Projects/Hz/datascience/iteration1/.venv/lib/python3.13/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/arison/Projects/Hz/datascience/iteration1/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 2170, in showtraceback
  File "/home/arison/Projects/Hz/datascience/iteration1/.venv/lib/python3.13/site-packages/IPython/core/

In [1]:
#loading the data with pandas
df = pd.read_csv('set9_it1.csv')
df.head(1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/arison/Projects/Hz/datascience/iteration1/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
  File "/tmp/ipykernel_4436/511186555.py", line 2, in <module>
    df = pd.read_csv('set9_it1.csv')
         ^^
NameError: name 'pd' is not defined. Did you mean: 'id'?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/arison/Projects/Hz/datascience/iteration1/.venv/lib/python3.13/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/arison/Projects/Hz/datascience/iteration1/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 2170, in showtraceback
  File "/home/arison/Projects/Hz/datascience/iteration1/.venv/lib/pyt

## Cleaning data

In [ ]:
# Checking for missing values
df.isna().sum()
# todo: decide what to do with the missing values 



DI                  0
TradeValue          0
RealmType           0
Enclave             0
LivingQuarters    117
ParcelSize          0
ParcelSizeUnit      0
dtype: int64

In [ ]:
# removes the duplicate rows in place
df.drop_duplicates(inplace=True)

- check for issues with types
- if ParcelSizeUnit is different we need to probably drop and then convert the ParcelSize so they have the same unit

In [ ]:
# todo: decide what to do with this fact
df[['ParcelSizeUnit']].squeeze().unique()

array(['sqft', 'sqm'], dtype=object)

RealType and Enclave probably have typos


In [ ]:
# Aparrently not this
print(df[['RealmType']].squeeze().unique())

# here we clearly have them and some things should be done about it 
sorted(df[['Enclave']].squeeze().unique())

# Cleaning capitalisation 

# 

['Meadowlands' 'GnomeBurough' 'ArcaneMarket' 'AetherIsles'
 'CitadelQuarter']


['BRIGHTFORGE',
 'Brightforge',
 'Cloudrest',
 'Crystalhollow',
 'Dragonspire',
 'Dragonspires',
 'Dreamweaver',
 'Duskwood',
 'EMBERFALL',
 'Emberfall',
 'Glimmerwood',
 'MYSTVALE',
 'Moonglade',
 'Mossheart',
 'Mystvale',
 'Nighthaven',
 'Oakenshade',
 'Petalbridge',
 'Ravenstone',
 'Riverbend',
 'SHADOWMERE',
 'Shadowmere',
 'Silverglen',
 'Starfall',
 'Stormhold',
 'Stormwatch',
 'Sunspire',
 'Sunspires',
 'THORNFIELD',
 'Thornfield',
 'Verdantia',
 'Windwhisper',
 'Wolfsbane',
 'Wyverncliff',
 'dragonspire',
 'emberfall',
 'glimmerwood',
 'shadowmere',
 'silverglen',
 'thornfield',
 'wyverncliff']

# Data understanding

Finding min and max value for each numerical row

In [ ]:
print("TradeValue: max: " +  str(df['TradeValue'].max()) + " min: " +  str(df['TradeValue'].min()))
print("LivingQuarters: max: " +  str(df['LivingQuarters'].max()) + " min: " +  str(df['LivingQuarters'].min()))
print("ParcelSize: max: " +  str(df['ParcelSize'].max()) + " min: " +  str(df['ParcelSize'].min()))

TradeValue: max: 755000.0 min: 113.5
LivingQuarters: max: 4339.0 min: 256.0
ParcelSize: max: 164660.0 min: -26400.0


Finding Mean, Median and mode for each row if applicable

In [ ]:
print("mean: ")
print(df[['TradeValue','LivingQuarters','ParcelSize']].mean())
print('\n')

print('median: ')
print(df[['TradeValue','LivingQuarters','ParcelSize']].median())
print('\n')

print("mode for numerics: ")
print(df[['TradeValue','LivingQuarters','ParcelSize']].mode().squeeze())
print('\n')
print("mode for strings: ")
print(df[['RealmType','Enclave']].mode().squeeze())


mean: 
TradeValue        181184.157291
LivingQuarters      1153.986955
ParcelSize          9826.716886
dtype: float64


median: 
TradeValue        160000.0
LivingQuarters      1110.0
ParcelSize          9328.5
dtype: float64


mode for numerics: 
TradeValue        135000.0
LivingQuarters       664.0
ParcelSize          9600.0
Name: 0, dtype: float64


mode for strings: 
RealmType    Meadowlands
Enclave      Dragonspire
Name: 0, dtype: object


Distributions

- Calculate Variance
- Show the distirution using a bell cerve graph 
- Comment and show on the graph skewedness of the distribution


Outliers

- Detect Outliers by highlighing area on a histogram
- Detect Outliers by comparing it to 3 times the stardard devation 
- Detect Outliers by using 1.5 * IQR also put a box graph for this one so it's visual as well 
- After all that decide what to do with the outliers and provide reasons 

Scatter Graphs


- Create scatter graph to see colleration between LivingQuarters (X) and TradeValue (Y)
- Create scatter graph to see colleration between ParcelSize (X) and TradeValue  (Y)
- Describe each graph's trends


Benchmark models

- Calculate the mean or median of TradeValue for combination of RealType and Enclave
- Calculate the MAE metric and MSE based on that  
- Split into Train and Testing sets 
- Do the MAE again... I think